<img src="https://1onjea25cyhx3uvxgs4vu325-wpengine.netdna-ssl.com/wp-content/themes/udacity_wp_1_8/images/Udacity_Logo_SVG_200x35.svg" style=" margin: 30px; height: 30px">



# Pneumonia Detection from Chest X-Rays

<img src="https://miro.medium.com/max/1400/1*rZ8wn3AO4CtzW_DGSQJjPg.png" style="height: 250px; width: 1000px">

### AI for Healthcare Nanodegree Program
**Project 1**

**Name**: Raghad Alharbi

**LinkedIn**: [Raghad-Alharbi](https://www.linkedin.com/in/raghad-alharbi/)

---
### Inference

In [1]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
import keras 
import glob
from skimage.transform import resize

Using TensorFlow backend.


In [2]:
mydicoms = glob.glob("*.dcm")
print('Number of DICOM Files: ',len(mydicoms))

Number of DICOM Files:  6


In [3]:
print('An over view of one of the DICOM files:')
dcm1 = pydicom.dcmread(mydicoms[0])
dcm1

An over view of one of the DICOM files:


(0008, 0016) SOP Class UID                       UI: Secondary Capture Image Storage
(0008, 0018) SOP Instance UID                    UI: 1.3.6.1.4.1.11129.5.5.110503645592756492463169821050252582267888
(0008, 0060) Modality                            CS: 'CT'
(0008, 1030) Study Description                   LO: 'No Finding'
(0010, 0020) Patient ID                          LO: '2'
(0010, 0040) Patient's Sex                       CS: 'M'
(0010, 1010) Patient's Age                       AS: '81'
(0018, 0015) Body Part Examined                  CS: 'CHEST'
(0018, 5100) Patient Position                    CS: 'PA'
(0020, 000d) Study Instance UID                  UI: 1.3.6.1.4.1.11129.5.5.112507010803284478207522016832191866964708
(0020, 000e) Series Instance UID                 UI: 1.3.6.1.4.1.11129.5.5.112630850362182468372440828755218293352329
(0028, 0002) Samples per Pixel                   US: 1
(0028, 0004) Photometric Interpretation          CS: 'MONOCHROME2'
(0028, 0010) Rows       

In [4]:
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data
def check_dicom(filename): 
    # todo
    print('Load file {} ...'.format(filename))
    ds = pydicom.dcmread(filename)       
    img = ds.pixel_array
    
    # we trained the model on X-ray images
    if ds.Modality != 'DX':
        print('Error: The examind image should be a Digital X-ray, and the provided image Modality is', ds.Modality);print()
        return None
    
    # we trained the model on only ages up to 100. 
    if int(ds.PatientAge) > 100:
        print('Error: The age of the patient should be a less than 100, and the provided DICOM is for age', ds.PatientAge);print()
        return None
    
    # we trained  the model on chest images
    if ds.BodyPartExamined != 'CHEST':
        print('Error: The examind image should be for a CHEST, and the provided image is for ', ds.BodyPartExamined);print()
        return None
    
    # we trained the model on AP and PA positions
    if ds.PatientPosition not in ['AP', 'PA']:
        print("Error: The patient position should be either 'AP' or 'PA', and the provided DICOM with a patient position of", ds.PatientPosition);print()
        return None
    
    return img
      
# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input
def preprocess_image(img,img_mean,img_std,img_size): 
    # todo
    proc_img = (img-img_mean) / img_std
    proc_img = resize(proc_img, img_size)
    
    return proc_img
    
# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path, weight_path):
    # todo
    with open(model_path, 'r') as json_file:
        model_file = json_file.read()
        model = keras.models.model_from_json(model_file)
        model.load_weights(weight_path)  
        
    return model

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, thresh): 
    # todo 
    prediction = np.where(model.predict(img) > thresh, 'Pneumonia', 'No_Pneumonia')
    
    return prediction 

In [6]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

model_path = "./my_model.json"
weight_path = "./xray_class_my_model.best.hdf5" 

IMG_SIZE= (1,224,224,3) # This might be different if you did not use vgg16
img_mean = 0 # loads the mean image value they used during training preprocessing
img_std = 255 # loads the std dev image value they used during training preprocessing

my_model = load_model(model_path, weight_path)
thresh = 0.41005385 #loads the threshold they chose for model classification 

# use the .dcm files to test your prediction
for i in test_dicoms:
    
    img = np.array([])
    img = check_dicom(i)
    
    if img is None:
        continue
        
    img_proc = preprocess_image(img,img_mean,img_std,IMG_SIZE)
    pred = predict_image(my_model,img_proc,thresh)
    print(pred);print()

Load file test1.dcm ...
[['Pneumonia']]

Load file test2.dcm ...
[['Pneumonia']]

Load file test3.dcm ...
[['Pneumonia']]

Load file test4.dcm ...
Error: The examind image should be for a CHEST, and the provided image is for  RIBCAGE

Load file test5.dcm ...
Error: The examind image should be a Digital X-ray, and the provided image Modality is CT

Load file test6.dcm ...
Error: The patient position should be either 'AP' or 'PA', and the provided DICOM with a patient position of XX

